In [1]:
import h5py
import cv2
import scipy.io as io
import PIL.Image as Image
import numpy as np
import os
import glob
from matplotlib import pyplot as plt
from scipy.ndimage.filters import gaussian_filter 
import scipy
import json
from torchvision import transforms
import torchvision.transforms.functional as F
from matplotlib import cm as CM
from model import SANet
import torch
%matplotlib inline

/home/mask/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from torchvision import datasets, transforms
transform=transforms.Compose([
                       transforms.ToTensor(),transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225]),
                   ])

In [3]:
root = './data/'

In [4]:
#now generate the ShanghaiA's ground truth
part_A_train = os.path.join(root,'part_A_final/train_data','images')
part_A_test = os.path.join(root,'part_A_final/test_data','images')
part_B_train = os.path.join(root,'part_B_final/train_data','images')
part_B_test = os.path.join(root,'part_B_final/test_data','images')
path_sets = [part_A_test]

In [5]:
img_paths = []
for path in path_sets:
    for img_path in glob.glob(os.path.join(path, '*.jpg')):
        img_paths.append(img_path)

In [6]:
model = SANet()

In [7]:
model = model.cuda()

In [8]:
checkpoint = torch.load('0model_best.pth.tar')

In [9]:
model.load_state_dict(checkpoint['state_dict'])

In [10]:
def combine_density(den,patch_idx,patch):
    patch_size = np.shape(patch)
    den_size = np.shape(den)
#     print(np.shape(den))
#     print(patch_size)
#     print(patch_idx)
    #axis in density map
    #x for verticle y for horizon
    dx = int(patch_idx/3)*int(patch_size[0]/2)
    dy = int(patch_idx%3)*int(patch_size[1]/2)

    dx_len = int(patch_size[0]*(3/4))
    dy_len = int(patch_size[1]*(3/4))
    #axis in patch
    px = 0
    py = 0 
    px_len = int(patch_size[0]*(3/4))
    py_len = int(patch_size[1]*(3/4))
    
    if dx!=0:
        dx = dx+int(patch_size[0]/4)
        px = int(patch_size[0]/4)
    if dx!=0 and dx+dx_len!=den_size[0]:
        dx_len = int(patch_size[0]/2)
        px_len = int(patch_size[0]/2)
    if dy!=0:
        dy = dy+int(patch_size[1]/4)
        py = int(patch_size[1]/4)
    if dy!=0 and dy+dy_len!=den_size[1]:
        dy_len = int(patch_size[1]/2)
        py_len = int(patch_size[1]/2)
        
#     print(dx,dy,dx_len,dy_len)
#     print(px,py,px_len,py_len)
        
    den[dx:dx+dx_len,dy:dy+dy_len] = patch[px:px+px_len,py:py+py_len]
    
        
        
    

In [11]:
std_transform=transforms.Compose([
   transforms.ToTensor(),transforms.Normalize(mean=[0.485, 0.456, 0.406],
                 std=[0.229, 0.224, 0.225]),
])
    
toTensor_transform = transforms.Compose([transforms.ToTensor()])

In [ ]:
mae = 0
for i in range(len(img_paths)):
    img = Image.open(img_paths[i]).convert('RGB')
    img = img.crop((0,0,int(img.size[0]/16)*16,int(img.size[1]/16)*16))
    gt_file = h5py.File(img_paths[i].replace('.jpg','.h5').replace('images','ground_truth'),'r')
    groundtruth = np.asarray(gt_file['density'])
    
    img = img.crop((0,0,int(img.size[0]/16)*16,int(img.size[1]/16)*16))
    target = groundtruth[0:img.size[1],0:img.size[0]]
    
    img = std_transform(img)
    img = transforms.ToPILImage()(img).convert('RGB') 

    den = np.zeros(np.shape(target))
    crop_size = (int(img.size[0]/2),int(img.size[1]/2))
    
    for j in range(9):
        dx = int((j/3)*crop_size[0])
        dy = int((j%3)*crop_size[1])
        patch = img.crop((dx,dy,crop_size[0]+dx,crop_size[1]+dy))
        patch = toTensor_transform(patch)
        patch = patch.cuda()

        with torch.no_grad():
            output = model(patch.unsqueeze(0))  
            output = output.squeeze().detach().cpu().numpy()
        combine_density(den,j,output)
        
    den_show = 255*den/np.max(den)
    cv2.imshow('Result', den_show)
    cv2.waitKey(0)

    mae += abs(np.sum(den)-np.sum(target))
    print(i,mae)
print(mae/len(img_paths))